In [ ]:
import numpy # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import os
for dirname, _, filenames in os.walk('../input'):
    for file in filenames:
        print(dirname + '/'+ file)

# RoBERTa Regression Model

In [ ]:
import pandas 
train = pandas.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pandas.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
train_df = train[['excerpt','target']]
submit = test[['id']]
test

In [ ]:
train_df.columns = ['text','labels']
test_df = list(test[['excerpt']].values.ravel())

In [ ]:
train_data = train_df[:2126]
eval_data = train_df[2126:]
test_data = test_df

In [ ]:
import os
import shutil
if os.path.exists('/kaggle/temp/wheels'):
    shutil.rmtree('/kaggle/temp/wheels')
if os.path.exists('/kaggle/working/wheels'):
    shutil.rmtree('/kaggle/working/wheels')
!unzip -q ../input/commonlit-readability-data/wheels.zip -d /kaggle/working/wheels 

In [ ]:
!pip install \
    --requirement ../input/commonlit-readability-data/requirements.txt \
    --no-index \
    --find-links /kaggle/working/wheels

In [ ]:
!conda install ../input/commonlit-readability-data/fsspec-2021.6.0-pyhd8ed1ab_0.tar.bz2 --offline --force-reinstall
!conda list | grep fsspec

In [ ]:
import fsspec
fsspec.__version__

In [ ]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args={
        "regression": True,
        "num_train_epochs":10
    },
    use_cuda = True
)

In [ ]:
model.train_model(train_data, eval_df=eval_data)

In [ ]:
import pickle
pickle.dump(model,open('roberta_model.pickle','wb'))
# model = pickle.load(open('../input/commonlit-readability/roberta_model.pickle','rb'))
predictions, raw_outputs = model.predict(test_data)
predictions

In [ ]:
submit['target'] = predictions
submit.to_csv('submission.csv',index = False)
submit